<h2><a href="https://leetcode.com/problems/tree-node">608. Tree Node</a></h2><h3>Medium</h3><hr><p>Table: <code>Tree</code></p>

<pre>
+-------------+------+
| Column Name | Type |
+-------------+------+
| id          | int  |
| p_id        | int  |
+-------------+------+
id is the column with unique values for this table.
Each row of this table contains information about the id of a node and the id of its parent node in a tree.
The given structure is always a valid tree.
</pre>

<p>&nbsp;</p>

<p>Each node in the tree can be one of three types:</p>

<ul>
	<li><strong>&quot;Leaf&quot;</strong>: if the node is a leaf node.</li>
	<li><strong>&quot;Root&quot;</strong>: if the node is the root of the tree.</li>
	<li><strong>&quot;Inner&quot;</strong>: If the node is neither a leaf node nor a root node.</li>
</ul>

<p>Write a solution to report the type of each node in the tree.</p>

<p>Return the result table in <strong>any order</strong>.</p>

<p>The result format is in the following example.</p>

<p>&nbsp;</p>
<p><strong class="example">Example 1:</strong></p>
<img alt="" src="https://assets.leetcode.com/uploads/2021/10/22/tree1.jpg" style="width: 304px; height: 224px;" />
<pre>
<strong>Input:</strong> 
Tree table:
+----+------+
| id | p_id |
+----+------+
| 1  | null |
| 2  | 1    |
| 3  | 1    |
| 4  | 2    |
| 5  | 2    |
+----+------+
<strong>Output:</strong> 
+----+-------+
| id | type  |
+----+-------+
| 1  | Root  |
| 2  | Inner |
| 3  | Leaf  |
| 4  | Leaf  |
| 5  | Leaf  |
+----+-------+
<strong>Explanation:</strong> 
Node 1 is the root node because its parent node is null and it has child nodes 2 and 3.
Node 2 is an inner node because it has parent node 1 and child node 4 and 5.
Nodes 3, 4, and 5 are leaf nodes because they have parent nodes and they do not have child nodes.
</pre>

<p><strong class="example">Example 2:</strong></p>
<img alt="" src="https://assets.leetcode.com/uploads/2021/10/22/tree2.jpg" style="width: 64px; height: 65px;" />
<pre>
<strong>Input:</strong> 
Tree table:
+----+------+
| id | p_id |
+----+------+
| 1  | null |
+----+------+
<strong>Output:</strong> 
+----+-------+
| id | type  |
+----+-------+
| 1  | Root  |
+----+-------+
<strong>Explanation:</strong> If there is only one node on the tree, you only need to output its root attributes.
</pre>

<p>&nbsp;</p>
<p><strong>Note:</strong> This question is the same as <a href="https://leetcode.com/problems/binary-tree-nodes/description/" target="_blank"> 3054: Binary Tree Nodes.</a></p>

In [1]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.getOrCreate()

# Hardcoded data for the Tree table (Example 1)
data = [
    (1, None),  # Node 1 is the root (no parent)
    (2, 1),     # Node 2 is a child of Node 1
    (3, 1),     # Node 3 is a child of Node 1
    (4, 2),     # Node 4 is a child of Node 2
    (5, 2)      # Node 5 is a child of Node 2
]

# Define the schema columns
columns = ["id", "p_id"]

# Create DataFrame from the hardcoded data
tree_df = spark.createDataFrame(data, schema=columns)

# Register the DataFrame as a temporary view for Spark SQL
tree_df.createOrReplaceTempView("Tree")

# Optional: Show the DataFrame
tree_df.show()


25/02/23 22:45:58 WARN Utils: Your hostname, Nahils-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.0.103 instead (on interface en0)
25/02/23 22:45:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/23 22:45:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+---+----+
| id|p_id|
+---+----+
|  1|NULL|
|  2|   1|
|  3|   1|
|  4|   2|
|  5|   2|
+---+----+



In [6]:
spark.sql("""
    SELECT t1.id,
           CASE
            WHEN (MAX(t1.p_id) IS NULL) THEN 'Root'
            WHEN (MAX(t1.p_id) IS NOT NULL) AND (MAX(t2.id) IS NOT NULL) THEN 'Inner'
            WHEN (MAX(t1.p_id) IS NOT NULL) AND (MAX(t2.id) IS NULL) THEN 'Child'
           END AS Type
          FROM Tree t1
          LEFT JOIN Tree t2 ON t1.id = t2.p_id
          GROUP BY t1.id
""").show()

+---+-----+
| id| Type|
+---+-----+
|  1| Root|
|  2|Inner|
|  3|Child|
|  4|Child|
|  5|Child|
+---+-----+

